

# Construindo um algoritmo de recomendação em pequena escala com Python

<p align="center">
    <img src="imgs/1032.jpg" width=700px >
</p>
<a href='https://br.freepik.com/vetores/bandeira'>Bandeira vetor criado por fullvector - br.freepik.com</a>

Já é senso comum que as Redes Sociais possuem algoritmos de recomendação que, ao mapear nossas preferências, nos indicam pessoas e assuntos que possuem uma alta chance de ser de nosso interesse. Contudo, o que poucos sabem é como esses sistemas funcionam por baixo dos panos. Pensando nisso, vamos construir alguns desses algoritmos, em pequena escala, auxiliados pelo primeiro capítulo do livro *Data Science do Zero, do autor Joel Grus*.

- - - 
# Motivação Hipotética: DataSciencester
O livro inicia com uma excelente motivação, a de ser contratado pela ***DataSciencester***, a Rede Social dos Cientistas de Dados. 

Logo no seu primeiro dia de trabalho o seu chefe te dá as boas-vindas com um belo de um problema para ser resolvido.

"Quero saber quem são os usuários mais populares de nossa plataforma"

Para te ajudar, ele te encaminha duas listas. A primeira com todos os usuários ativos e a segunda com tuplas que indicam amizades entre os cientistas, representada pelos seus respectivos IDs.

In [5]:
users = [ 
         {"id": 0, "name": "Hero"},
         {"id": 1, "name": "Dunn"},
         {"id": 2, "name": "Sue"},
         {"id": 3, "name": "Chi"},
         {"id": 4, "name": "Thor"},
         {"id": 5, "name": "Clive"},
         {"id": 6, "name": "Hicks"},
         {"id": 7, "name": "Devin"},
         {"id": 8, "name": "Kate"},
         {"id": 9, "name": "Klein"}
]

friendships = [(0,1),(0,2),(1,2),(1,3),(2,3),(3,4),
              (4,5),(5,6), (5,7), (6,8), (7,8), (8,9)]
          # A tupla(0,1) indica que o cientista de dados com id 0(Hero)
          # é amigo do com id 1(Dunn)

Por maior que seja a tentação de ir manipulando o código, vamos nos questionar sobre o problema. Este é um passo necessário, pular ele provavelmente implicará em retrabalho.
- O que significa um usuário ser popular para a plataforma? Ou melhor, como podemos medir a popularidade?
- Uma forma de indicar isso é pelo número de conexões que cada usuário possui, aqueles que possuem mais conexões são os mais "populares", ou seja, os mais relevantes para a plataforma. 

Para descobrir o número de conexões de cada usuário, vamos criar uma lista de amigos para cada usuário na lista acima(**Passo 1**), a fim de calcular o número médio de conexões dos usuários(**Passo 2**) e , por fim, elencar uma lista com a quantidade de conexões que cada usuário possui(**Passo 3**).

### Passo 1 - Criando uma lista de amigos para cada usuário na lista acima

In [6]:
# Cada usuário está recebendo uma lista de amigos vazia.
for user in users: 
  user["friends"] = []

# Vamos povoar a lista com dados de friendships
for i, j in friendships:
  users[i]["friends"].append(users[j])# adiciona j como amigo de i
  users[j]["friends"].append(users[i])# adiciona i como amigo de j

### Passo 2 - Calculando o número médio de conexões

In [9]:
def number_of_friends(user):
  # quantos amigos o usuário tem?
  return len(user["friends"])

total_connections = sum(number_of_friends(user) for user in users)  # 24
 
num_users = len(users)  # tamanho da lista de usuários
avg_connections = total_connections / num_users  # 2.4 = Valor médio de conexões
 

### Passo 3 - Encontrando as pessoas mais conectadas
São as que possuem o maior número de amigos.

In [10]:
num_friends_by_id = [(user["id"], number_of_friends(user)) 
                      for user in users]
sorted(num_friends_by_id, key=lambda tup: tup[1], reverse = True ) #ordenando tuplas com base no segundo elemento(posição 1)
# A primeira posição é o ID do usuário e a segunda a quantidade de amigos.


[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1)]

O user de id 1 possui 3 amigos, o de id 0 possui 2, etc.
- - -

Nosso problema foi resolvido, mas, como queremos impressionar o chefe, vamos entregar algo mais. Vamos implementar na *DataSciencester* a funcionalidade ***Cientistas de Dados que talvez você conheça***.

Para isso, vamos criar uma função que nos retorne os amigos de cada usuário.

In [11]:
def friends_of_friend_ids_bad(user):
  return [ foaf["id"] ##foaf é uma abreviatura para friends of friends
          for friend in user["friends"]
          for foaf in friend["friends"]]
          
friends_of_friend_ids_bad(users[0])

[0, 2, 3, 0, 1, 3]

Não pareceu tão eficiente, pois ao checar os amigos do usuário 0 repetiu-se alguns Ids, inclusive o próprio 0.

Vamos então refinar o código.


In [13]:
from collections import Counter

def not_the_same(user, other_user):
  # dois usuários não são o mesmo se possuem ids diferentes
  return user["id"] != other_user["id"]

def not_friends(user, other_user):
  # other_user não é um amigo se não está em user["friends"]:
  # isto é, se não é not_the_same com todas as pessoas em user["friends"]
  return all (not_the_same(friend, other_user)
              for friend in user["friends"])

def friends_of_friend_ids(user):
  return Counter( foaf["id"]
                 for friend in user["friends"]
                 for foaf in friend["friends"]
                 if not_the_same(user, foaf)
                 and not_friends(user, foaf))
  
print(friends_of_friend_ids(users[3])) 

Counter({0: 2, 5: 1})


O usuário de id 3 tem dois amigos em comum com 0 e um com 5.

### Agora que o negócio vai ficar bom
Seu chefe gostou tanto do seu trabalho, que, antes de finalizar o dia, ele te pediu para fazer um Algoritmo para ***Identificar Usuários com Interesses semelhantes***

Depois de perguntar por aí, você conseguiu pôr a mão nos dados de interesses de cada usuário. Eles estão ordenados como uma lista de pares(user_id, interest):

In [14]:
interests = [
(0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
(0, "Spark"), (0, "Storm"), (0, "Cassandra"),
(1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
(1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
(2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
(3, "statistics"), (3, "regression"), (3, "probability"),
(4, "machine learning"), (4, "regression"), (4, "decision trees"),
(4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
(5, "Haskell"), (5, "programming languages"), (6, "statistics"),
(6, "probability"), (6, "mathematics"), (6, "theory"),
(7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
(7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
(8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
(9, "Java"), (9, "MapReduce"), (9, "Big Data")
]


É fácil construir uma função que encontre usuários com o mesmo interesse:

In [16]:

def data_scientists_who_like(target_interest): 
  return[ user_id
            for user_id, user_interest in interests
            if user_interest == target_interest]
print(data_scientists_who_like("Big Data"))

[0, 8, 9]


Funciona, mas não é tão eficiente porque a lista inteira deve ser examinada a cada busca. Imagina só o trabalhão que daria numa lista com milhões de usuários e interesses. A empresa não iria gostar do custo computacional e financeiro gerado.

Por isso, seria melhor construir um índice de interesses para usuários:

In [17]:
from collections import defaultdict

# as chaves são os interesses, os valores são as listas de user_ids com esses interesses.
users_ids_by_interest = defaultdict(list)

for user_id, interest in interests:
  users_ids_by_interest[interest].append(user_id)

E outra de usuários para interesses

In [25]:
interests_by_users_id = defaultdict(list)

for user_id, interest in interests:
  interests_by_users_id[user_id].append(interest)

Agora fica fácil descobrir quem possui os maiores interesses em comum com um certo usuário:
- Itera sobre os interesses do usuário
- Para cada interesse, itera sobre os outros usuários com aquele interesse
- Mantém a contagem de quantas vezes vemos cada outro usuário

In [27]:
def most_common_interests_with(user):
  return Counter(interested_user_id
                 for interest in interests_by_users_id[user["id"]]
                 for interested_user_id in users_ids_by_interest[interest]
                 if interested_user_id != user["id"])
  
  
most_common_interests_with(users[5])

Counter({0: 1, 2: 1, 3: 2, 9: 1})

O usuário 5 possui dois interesses em comum com o usuário 3. Possui 1 interesse em comum com 0, com 2 e com 9.

Com base nisso, poderia ser criada a funcionalidade: **Cientistas de Dados que você deveria conhecer**.

### Considerações Finais:

O presente artigo foi baseado no Capítulo 1 do livro Data Science do Zero, do autor Joel Grus, ele foi usado como fonte principal de contextualização e de códigos.

Você tem alguma observação sobre alguma funcionalidade que eu poderia usar? Ou uma boa prática que eu não levei em consideração? Ficarei feliz em ler essas observações aqui nos comentários.